<a href="https://colab.research.google.com/github/harshitadd/AI4BharatTranslation/blob/main/LaBSEAligner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers

     |████████████████████████████████| 71kB 5.0MB/s 
     |████████████████████████████████| 1.8MB 15.5MB/s 
     |████████████████████████████████| 1.2MB 38.6MB/s 
     |████████████████████████████████| 2.9MB 69.8MB/s 
     |████████████████████████████████| 890kB 20.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=f92bd7930af685566a3be1d70774a436a0d7f20259e0011bd6f334c7e3152b00
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a48c10b25971ea74660feadaa53f983abad7b38a2325379f78dcc1c96e3bde81
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [2]:
!pip install scipy

In [3]:
!pip install indic-nlp-library

In [4]:
import sys
from indicnlp.tokenize import indic_tokenize
from sentence_transformers import SentenceTransformer
import numpy as np
from scipy.spatial import distance
from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
# model = AutoModel.from_pretrained("sentence-transformers/LaBSE")
model = SentenceTransformer('LaBSE')



100%|██████████| 1.75G/1.75G [00:30<00:00, 58.0MB/s]


In [5]:
!pip install sentence-splitter

     |████████████████████████████████| 51kB 3.9MB/s 


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
with open('/content/drive/MyDrive/eng_2.txt','r') as file:
  content = file.read()

In [12]:
# Only for English
import re 
from sentence_splitter import SentenceSplitter, split_text_into_sentences
content = content.replace('\n',' ')
splitter = SentenceSplitter(language='en')
sentences = splitter.split(content)

# final = ''
# for s in sentences:
# 	final+=s
# 	final+='\n'
# sentence_count=len(sentences)
# print(sentence_count)


In [17]:
len(sentences)

813

In [19]:
temp = sentences[0:20]

In [15]:
def phrase_aligner(src, tgtp):
      '''
      This function is meant to align src phrases with best possible tgt phrase using LABSE model
      '''
      out = {}
      aligned_phrases = {}
      print("Performing phrase alignenment using LABSE")
      print(src,tgtp)
      src_phrases, tgt = src, tgtp
      
      for src_phrase in src_phrases:
          length_src_phrase = len(src_phrase.split())           
          tgt_token_list = split_tgt(length_src_phrase,tgt)
          embeddings_src_phrase, embeddings_tgt_tokens = generate_embeddings([src_phrase],tgt_token_list)
          alignments = get_target_sentence(embeddings_tgt_tokens, embeddings_src_phrase, length_src_phrase)
      
          if alignments is not None and alignments[2] is "MATCH":
              aligned_phrases[src_phrase] = tgt_token_list[alignments[0]]
          elif alignments is not None and alignments[2] is "NOMATCH": 
              print("No exact match found for:{} . Possible alignment {}".format(src_phrase,tgt_token_list[alignments[0]]))  
                      
      print("Aligned Phrases: {}".format(aligned_phrases))
      out = {"tgt":tgt,"src_phrases":src_phrases,"aligned_phrases":aligned_phrases}     

      return out

def split_tgt(length_src_phrase,tgt):
  tgt_token_list = list()
  tokenised_tgt = tgt.split()
  tgt_token_list = [tokenised_tgt[i:i+length_src_phrase] for i in range(len(tokenised_tgt)) if (i + length_src_phrase) <= len(tokenised_tgt)]
  tgt_token_list_plus = [tokenised_tgt[i:i+length_src_phrase+1] for i in range(len(tokenised_tgt)) if (i + length_src_phrase+1) <= len(tokenised_tgt)]
  tgt_token_list_minus = [tokenised_tgt[i:i+length_src_phrase-1] for i in range(len(tokenised_tgt)) if (i + length_src_phrase-1) <= len(tokenised_tgt) and 
                          length_src_phrase != 1]
  tgt_token_list = tgt_token_list + tgt_token_list_plus + tgt_token_list_minus
  tgt_token_list = [" ".join(j) for j in tgt_token_list]
  return tgt_token_list
      
def generate_embeddings(input_1, input_2):
  '''
  Generate LABSE embeddings
  Note: Inputs are array of strings
  '''           
  embeddings_input_1 = model.encode(input_1,show_progress_bar=True)
  embeddings_input_2 = model.encode(input_2,show_progress_bar=True)    
  print("LABSE embedding generation finished")
  return embeddings_input_1, embeddings_input_2
  
def get_target_sentence(target_embeddings, source_embedding, length_src_phrase):
  '''
  Calculate cosine similarity using scipy distance method
  '''
  distances = distance.cdist(source_embedding, target_embeddings, "cosine")[0]
  min_index = np.argmin(distances)
  min_distance = 1 - distances[min_index]
  print("Match score: {}".format(min_distance))
  if min_distance >= 0.5:
      return min_index, min_distance, "MATCH"
  else:
      return min_index, min_distance, "NOMATCH"     
      


In [ ]:
print(content)

In [21]:
tgtp = '''ஆன்மாவின்‌ முன்‌ தலைவணங்கி, 2018-2019 ஆம்‌ ஆண்டிற்கான
நிதிநிலை அறிக்கையை இந்தப்‌ பேரவையின்‌ முன்‌ சமர்ப்பிக்க
விழைகிறேன்‌.

2. மக்களின்‌ உள்ளங்களில்‌ நீங்கா இடம்‌ பெற்று, அவர்களின்‌
பேரன்பைப்‌ பெற்ற மாபெரும்‌ தலைவியாக விளங்கிய புரட்சித்‌ தலைவி
அம்மா அவர்கள்‌ தொடங்கி வைத்த அனைத்து திட்டங்களையும்‌
தொடர்ந்து செயல்படுத்துவதில்‌ இந்த அரசு உறுதியாக உள்ளது.
தொலைநோக்குப்‌ பார்வையும்‌, மதிநுட்பமும்‌ வாய்ந்த புரட்சித்‌ தலைவி
அம்மா அவர்களின்‌ “அமைதி, வளம்‌, வளர்ச்சி! என்ற கொள்கை முழக்கம்‌,
இந்த மாநிலத்தை முன்னேற்றப்‌ பாதையில்‌ கொண்டு செல்வதற்கான
வழிகாட்டும்‌ அடிப்படை கொள்கையாக தொடர்ந்து விளங்கும்‌. பொது
சுகாதாரம்‌, கல்வி, சமூகப்‌ பாதுகாப்பு போன்ற துறைகளில்‌
செயல்படுத்தப்பட்டு வரும்‌ பல்வேறு திட்டங்கள்‌, நமது மாநிலத்தின்‌
மனிதவள மேம்பாட்டுக்‌ குறியீடுகளை உயர்த்தி, நாட்டிலேயே
சமூகநலத்தைப்‌ பேணிப்‌ போற்றும்‌ முன்மாதிரி மாநிலமாகத்‌ தமிழ்நாட்டை
உருவாக்கியுள்ளது என்பதை யாராலும்‌ மறுக்க முடியாது. கடல்‌ அலை
உயரும்போது அனைத்துப்‌ படகுகளும்‌ ஒருசேர உயர்வது போல்‌,
மாநிலத்தின்‌ துரிதமான பொருளாதார வளர்ச்சி, அனைத்துக்‌
குடும்பங்களின்‌ பொருளாதார நிலையையும்‌ ஒருசேர உயர்த்தும்‌ என்பதில்‌
ஐயமில்லை. எனினும்‌, சமுதாயத்தில்‌ அனைத்து மக்களின்‌ வாழ்வையும்‌
மேம்படுத்தி, ஏற்றத்தாழ்வுகளைக்‌ களைந்து, பொருளாதாரச்‌ சம
நிலையைக்‌ கொண்டு வருவதற்காக, பல்வேறு நலத்திட்டங்களை,
குறிப்பாக ஏழை எளிய மக்களின்‌ மேம்பாட்டிற்கான திட்டங்களை
இந்த அரசு தொடர்ந்து செயல்படுத்தும்‌. எனவே, பொருளாதார
மேம்பாட்டிற்கு பெரும்‌ ஊக்கம்‌ தரும்‌ அதே வேளையில்‌, நலத்திட்டப்‌
பணிகளும்‌ இந்த அரசின்‌ வளர்ச்சிக்‌ கொள்கையில்‌ தொடர்ந்து

முக்கியத்துவத்தைப்‌ பெறும்‌.

3. பல சவாலான சூழ்நிலைகளுக்கிடையே, இந்த வரவு-செலவுத்‌

திட்ட அறிக்கை சமர்ப்பிக்கப்படுகிறது. பணமதிப்பிழப்பு, பொருட்கள்‌

மற்றும்‌ சேவைகள்‌ வரி விதிப்பு முறை நடைமுறைக்கு வந்த பிறகு

ஏற்பட்டுள்ள பெருத்த மாற்றங்களைக்‌ கடந்து, தொடர்ந்து முன்னேற

வேண்டிய முக்கியக்‌ கட்டத்தில்‌ நமது பொருளாதாரம்‌ உள்ளது. இந்த

முக்கியத்‌ தருணத்தில்‌, மாநிலம்‌ சந்தித்து வரும்‌ பல்வேறு சவால்கள்‌:

,
க

5,
te

o,
te

°,
og

°,
8

முதலீடுகளை ஈர்த்து, அதிக வேலைவாய்ப்புகளை உருவாக்குதல்‌

இளைஞர்களுக்குத்‌ திறன்‌ பயிற்சி அளித்து,

வேலைவாய்ப்பிற்கான தகுதியை அவர்கள்‌ பெறச்‌ செய்தல்‌

விவசாய உற்பத்தித்‌ திறனையும்‌, விவசாயிகளின்‌ வருவாயையும்‌
பெருக்குதல்‌

நகர்ப்புரங்களில்‌ அடிப்படை வசதிகளையும்‌, உட்கட்டமைப்பையும்‌

மேம்படுத்துதல்‌

அனைவருக்கும்‌ வீட்டுவசதி என்ற இலக்கை அடைய

ஏழைகளுக்கு வீட்டுவசதி வழங்குதல்‌

நீர்நிலை உள்ளிட்ட இயற்கை ஆதாரங்களை பாதுகாத்துப்‌

பராமரித்தல்‌
நகர்ப்புரங்களிலும்‌, ஊரகப்‌ பகுதிகளிலும்‌ வறுமை ஒழிப்பு

கல்வி, சுகாதாரம்‌, மருத்துவ வசதிகளின்‌ தரத்தை மேம்படுத்தி,
அவற்றை அனைவரும்‌ எளிதாகப்‌ பெற வழிவகை செய்தல்‌
பருவநிலை மாற்றத்தால்‌ ஏற்படும்‌ தாக்கத்தைக்‌ குறைப்பதுடன்‌,
கடல்‌ அரிப்பிலிருந்து கடலோரப்‌ பகுதிகளைப்‌ பாதுகாத்தல்‌

குறைந்து வரும்‌ மத்திய வரிப்‌ பகிர்வு, சொந்த வரி, வரியல்லாத
வருவாய்‌ வரவினங்களில்‌ உள்ள தற்போதைய சூழ்நிலையை

சமாளித்து, நிதிநிலையை திறம்பட மேலாண்மை செய்தல்‌

இந்த பத்து முக்கியச்‌ சவால்களையும்‌ எதிர்கொண்டு
பொருளாதார வளர்ச்சியை ஊக்குவித்து, தமிழ்நாட்டை வளமான
மாநிலமாக மாற்றச்‌ செய்வதற்கான உத்திகளை உள்ளடக்கி இந்த

நிதிநிலை அறிக்கை தயாரிக்கப்பட்டுள்ளது.
பொருளாதார வளர்ச்சி

மாண்புமிகு பேரவைத்‌ தலைவர்‌ அவர்களே,

4. மாநிலத்தின்‌ பொருளாதாரத்தில்‌ நிலவி வந்த மந்தநிலையில்‌
மாற்றம்‌ ஏற்பட்டு, 2017-2018 ஆம்‌ ஆண்டில்‌ பொருளாதார வளர்ச்சி
ஒரளவு உயர்ந்துள்ள நிலையில்‌, 2018-2019 ஆம்‌ ஆண்டில்‌ இது மேலும்‌
உயரும்‌ எனக்‌ கருதப்படுகிறது. தமிழ்நாட்டின்‌ மாநில மொத்த
உற்பத்தியின்‌ வளர்ச்சி விகிதம்‌, நிலையான விலைகளின்‌ அடிப்படையில்‌,
2017-2018 ஆம்‌ ஆண்டில்‌ 8.03 சதவீதமாக இருக்கும்‌ என
கணிக்கப்பட்டுள்ளது. மாநிலப்‌ பொருளாதாரத்தில்‌ காணப்படும்‌
இத்தகைய சாதகமான சூழ்நிலையில்‌, முதன்மைத்‌ துறையில்‌ இந்த அரசு
தொடர்ந்து செய்து வரும்‌ நீடித்த முதலீடுகளாலும்‌, மாநிலத்தில்‌ தொழில்‌
துறை முதலீடுகளை ஊக்குவிப்பதற்கு அரசால்‌ மேற்கொள்ளப்பட்டு வரும்‌
நடவடிக்கைகளாலும்‌, 2018-2019 ஆம்‌ ஆண்டில்‌ மாநிலப்‌ பொருளாதார
வளர்ச்சி விகிதம்‌ 9 சதவீதமாக உயரும்‌ எனக்‌ கருதப்படுகிறது.
பொருட்கள்‌ மற்றும்‌ சேவைகள்‌ வரி

5. பொருட்கள்‌ மற்றும்‌ சேவைகள்‌ வரி (6851) விதிப்பு முறை,
2017 ஆம்‌ ஆண்டு ஜூலை மாதம்‌ 1ஆம்‌ தேதி முதல்‌, நாட்டில்‌
அறிமுகப்படுத்தப்பட்டுள்ளதை மாண்புமிகு உறுப்பினர்கள்‌ அறிவீர்கள்‌.
புதிய வரி விதிப்பு முறையை நடைமுறைப்படுத்தும்போது மாநிலங்களுக்கு
ஏற்படக்கூடிய வரி வருவாய்‌ இழப்பினை ஐந்தாண்டுகள்‌ வரை
மத்திய அரசு ஈடுசெய்யும்‌. பொருட்கள்‌ மற்றும்‌ சேவைகள்‌ வரியின்‌ கீழ்‌,
2017 ஆம்‌ ஆண்டு ஜூலை மாதம்‌ முதல்‌ பிப்ரவரி 2018 வரை 632 கோடி
ரூபாயினை இழப்பீடாக தமிழ்நாடு மத்திய அரசிடமிருந்து பெற்றுள்ளது.
கடந்த எட்டுமாத காலத்தில்‌, 14 சதவீத வளர்ச்சி விகிதத்துடன்‌
எதிர்பார்க்கப்படும்‌ உறுதி செய்யப்பட்ட வருவாய்க்கும்‌, கிடைக்கப்பெற்ற
வருவாய்க்கும்‌ இடையேயான இடைவெளி கணிசமாகக்‌ குறைந்து
வருவதால்‌, பொருட்கள்‌ மற்றும்‌ சேவைகள்‌ வரி விதிப்பு முறையினால்‌
நமது மாநிலம்‌ நல்ல பலனைப்‌ பெற்றுள்ளது என்பதைத்‌ தெரிவித்துக்‌
கொள்கிறேன்‌.
'''

In [23]:
# with open('/content/drive/MyDrive/tam_0_2.txt','r') as file:
#   content = file.read()
src = temp 
# tgtp = content
res = phrase_aligner(src, tgtp)

Performing phrase alignenment using LABSE
['Speech of Thiru D. Jayakumar, Hon’ble Minister for Fisheries, Finance and _ Personnel’ and Administrative Reforms, Government of Tamil Nadu, presenting the Budget for the year 2017-2018 to the _ Legislative Assembly on 16" March, 2017 Hon’ble Speaker Sir, QT MT 2 WHFS! 9 wiiihS YSipsvevrrd Qurrett pg) H\\HuG sett Med (Gps 233) Save praise alone that soars on high, Nought lives on earth that shall not die (Kural 233) There is nothing that stands forth in the world imperishable, except fame, exalted in solitary greatness.', 'This eternal wisdom of the Divine Poet Thiruvalluvar was aptly fulfilled through the life of our revered leader the late Chief Minister Puratchi Thalaivi J Jayalalithaa.', 'Today, her fame has reached worldwide through her commitment to the well being of the people whom she loved and for the progress of the State she made into a model welfare State in the country.', 'We are proud, honoured and privileged to have lived durin

KeyboardInterrupt: ignored

In [ ]:
print(res)

{'tgt': 'भारत का संविधान अप्रैल,  को यथाविद्यमान संसद्\u200c द्वारा समयसमय पर यथा संशोधित भारत केसंविधान के पाठ को उद्धृत करता है । संसद्\u200c द्वारा, संविधान (एक सौ तीनवां संशोधन) अधिनियम, तक और उसको सम्मिलित करते हुए, किए गए सभी संशोधन, इस संस्करण मैं सम्मिलित किए गएहै । पाठ के नीचे दिए गए पाद टिप्पण, संविधान संशोधन अधिनियमों, जिसके द्वारा ऐसे संशोधन किएगए है, को उपदर्शित करता है ।', 'src_phrases': ['This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time', ' All amendments made by the Parliament up to and including the Constitution (One Hundred and Third Amendment) Act, 2019 are incorporated in this edition', 'The foot notes below the text indicate the Constitution Amendment Acts by which such amendments have been made', ''], 'aligned_phrases': {'This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time': 'को यथाविद्यमान संसद्\u20

In [ ]:
print(res['aligned_phrases'])

{'This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time': 'को यथाविद्यमान संसद्\u200c द्वारा समयसमय पर यथा संशोधित भारत केसंविधान के पाठ को उद्धृत करता है । संसद्\u200c द्वारा, संविधान (एक सौ', ' All amendments made by the Parliament up to and including the Constitution (One Hundred and Third Amendment) Act, 2019 are incorporated in this edition': 'है । संसद्\u200c द्वारा, संविधान (एक सौ तीनवां संशोधन) अधिनियम, तक और उसको सम्मिलित करते हुए, किए गए सभी संशोधन, इस संस्करण मैं सम्मिलित', 'The foot notes below the text indicate the Constitution Amendment Acts by which such amendments have been made': 'पाठ के नीचे दिए गए पाद टिप्पण, संविधान संशोधन अधिनियमों, जिसके द्वारा ऐसे संशोधन किएगए है, को उपदर्शित करता', '': ''}


In [ ]:
for key in res['aligned_phrases']:
  print(key)
  print(res['aligned_phrases'][key])
  

This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time
को यथाविद्यमान संसद्‌ द्वारा समयसमय पर यथा संशोधित भारत केसंविधान के पाठ को उद्धृत करता है । संसद्‌ द्वारा, संविधान (एक सौ
 All amendments made by the Parliament up to and including the Constitution (One Hundred and Third Amendment) Act, 2019 are incorporated in this edition
है । संसद्‌ द्वारा, संविधान (एक सौ तीनवां संशोधन) अधिनियम, तक और उसको सम्मिलित करते हुए, किए गए सभी संशोधन, इस संस्करण मैं सम्मिलित
The foot notes below the text indicate the Constitution Amendment Acts by which such amendments have been made
पाठ के नीचे दिए गए पाद टिप्पण, संविधान संशोधन अधिनियमों, जिसके द्वारा ऐसे संशोधन किएगए है, को उपदर्शित करता


